In [3]:
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn
import sklearn.tree
import sklearn.ensemble
from sklearn.cross_validation import cross_val_score
from sklearn import preprocessing as pp
%matplotlib inline

/Users/lokotochek/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [4]:
data_train_read = pd.read_csv("train.csv")
data_test_read = pd.read_csv("test.csv")

In [5]:
data_train = data_train_read.copy()

In [6]:
data_test = data_test_read.copy()

In [7]:
table = data_train.corr()

In [8]:
#print table.head(12)
#print data.columns[[10, 11]]

tv = table.values
edges = []
for i in range(len(tv)):
    for j in range(i + 1, len(tv) - 1):
        if np.abs(tv[i][j]) > .97:
            edges.append([i, j])

print len(edges)

202


In [9]:
vertices = {}
for e in edges:
    vertices[e[0]] = True
    vertices[e[1]] = True
    
vertices = vertices.keys()
do_not_del = [False] * len(vertices)
power = [0] * len(vertices)
to_del = []
for edge in edges:
    power[vertices.index(edge[0])] += 1
    power[vertices.index(edge[1])] += 1
for i in range(len(vertices)):
    if power[i] == 1:
        to_del.append(vertices[i])
        
to_del = data_train.columns[to_del]

In [10]:
for cur in to_del:
    del data_train[cur]
    del data_test[cur]

In [15]:
X_train = data_train[data_train.columns[1:len(data_train.columns) - 1]]
y_train = data_train['TARGET']

In [47]:
clf = xgb.XGBRegressor(nthread=4, 
                       max_depth=8, 
                       n_estimators=30, 
                       subsample=0.5, 
                       learning_rate=0.006, 
                       seed=1337)

In [48]:
scores = cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=3)
print scores.mean()

0.828266253987


In [35]:
clf.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.006, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=4,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1337, silent=True, subsample=0.5)

In [36]:
X_test = data_test[data_test.columns[1:]]

In [37]:
y_test = clf.predict(X_test)

In [40]:
trash_count = 0
for i in range(len(y_test)):
    if y_test[i] < 0:
        y_test[i] = 0
        trash_count += 1
    if y_test[i] > 1:
        y_test[i] = 1
        trash_count += 1
print trash_count
ans = pd.DataFrame(index=data_test['ID'], data={'TARGET': y_test})
ans.to_csv('kek.csv')

0


# normalize

In [16]:
min_max_scaler = pp.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)

In [17]:
pd.DataFrame(X_train_minmax).describe()

,0,1,2,3,4,5,6,7,8,9,...,263,264,265,266,267,268,269,270,271,272
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000
mean,0.998239,0.282129,0.000411,0.005615,0.005685,0.000432,0.000585,0.000063,0.000086,0.000384,...,0.000025,0.000159,0.000067,0.000088,0.000096,0.000072,0.000075,0.000112,0.000142,0.005087
std,0.039024,0.129565,0.007689,0.026328,0.025982,0.011308,0.013883,0.004637,0.005532,0.011565,...,0.004348,0.009117,0.005590,0.005641,0.005867,0.004593,0.005995,0.005929,0.007169,0.008292
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.999764,0.180000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002846
50%,0.999764,0.230000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004596
75%,0.999764,0.350000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005156
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
